In [ ]:
from github import Github


def create_file_level_comment(token, repo_name, pull_number, commit_id, path, body):
    """
    Create a general file-level comment on a pull request using PyGithub.

    Args:
    token (str): GitHub access token.
    repo_name (str): Repository name including owner ('owner/repo').
    pull_number (int): Number of the pull request.
    commit_id (str): SHA of the commit for commenting.
    path (str): File path relative to the repository root.
    body (str): Text of the comment.

    Returns:
    None
    """
    g = Github(token)
    repo = g.get_repo(repo_name)
    pull = repo.get_pull(pull_number)
    # Get the diff of the pull request to find the last position to place the comment
    files = pull.get_files()
    for file in files:
        if file.filename == path:
            # Place the comment on the last position of the file changes if possible
            if file.patch:
                lines = file.patch.split("\n")
                position = (
                    len(lines) - 1
                )  # assuming last line is related to the file change
                pull.create_review_comment(body, commit_id, path, position)
                return
    # Fallback if no suitable line found in diff
    print("No suitable line found in diff to place the comment.")

In [ ]:
from llamabot import SimpleBot
from llamabot.prompt_manager import prompt
import json5 as json


@prompt
def summarize_file_diff(diff):
    """Based on the following diff:

    {{ diff }}

    Write for me a summary of changes in this file in plain English.
    Guess the intent of the changes in the file, and connect those intents to the changes.
    Follow the JSON schema below:

    {
        "intent": <intent>,
        "summary": <summary>,
        "bullet_points": [<bullet_point1>, <bullet_point2>,...]
    }
    """

In [ ]:
@prompt
def comment_template(intent, summary, bullet_points):
    """
    ## Intent

    {{ intent }}

    ## Summary of Changes

    {{ summary }}

    ## Details

    {% for bullet_point in bullet_points %}- {{ bullet_point }}
    {% endfor %}

    _Change summary automatically written by LlamaBot._
    """


print(comment_template(changes["intent"], changes["summary"], changes["bullet_points"]))

In [ ]:
def get_first_diff_hunk(patch):
    """
    Extracts the first diff hunk from a unified diff string.

    Args:
    patch (str): The unified diff string of the file.

    Returns:
    str: The first diff hunk found.
    """
    lines = patch.split("\n")
    hunk = []
    in_hunk = False

    for line in lines:
        if line.startswith("@@"):
            if in_hunk:
                # Finish the current hunk when a new one starts
                break
            else:
                # Start a new hunk
                in_hunk = True
                hunk.append(line)
        elif in_hunk:
            hunk.append(line)

    return "\n".join(hunk)

In [ ]:
def top_line_number(diff_hunk):
    # Return the position of the first line in the diff_hunk.

    # Line in file that the hunk starts at:
    # Parse diff_hunk first line for the line number in original file.
    original_line_number = int(diff_hunk.split(" ")[2].split(",")[0].strip("+"))
    return original_line_number


# Example usage
diff_hunk = """
@@ -10,4 +10,6 @@
 context line
-removed line
+added line
+another added line
 context line
"""

top_line_number(diff_hunk)

In [ ]:
for commit in pull.get_commits():
    print(commit.files)

In [ ]:
commit.files

In [ ]:
from github.File import File
from github.PullRequest import PullRequest
from github.Commit import Commit


def get_latest_commit_for_file(pull: PullRequest, file: File) -> Commit:
    """Get the latest commit for a given file in a pull request.

    :param pull: The pull request object.
    :param file: The file object.
    """
    for commit in pull.get_commits():
        for file2 in commit.files:
            if file2.filename == file.filename:
                break
    return commit